In [9]:
#installing all the required libraries
!pip install langchain_groq langchain_core langchain_community
!pip install pypdf
!pip install chromadb
!pip install sentence_transformers
!pip install gradio
!pip install vaderSentiment


In [13]:
import os
import json
from datetime import datetime
from langchain_groq import ChatGroq
import gradio as gr
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter


# Initialized VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

def initialize_llm():
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",  # Replace with groq API key
        model_name="llama-3.3-70b-versatile"
    )
    return llm

def create_vector_db():
    loader = DirectoryLoader("/content/data/", glob="*.pdf", loader_cls=PyPDFLoader) #load mental_health_doc from data folder
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")
    vector_db.persist()
    print("ChromaDB created and data saved")
    return vector_db

def setup_qa_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt_templates = """You are a compassionate mental health chatbot. Respond thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot: """
    PROMPT = PromptTemplate(template=prompt_templates, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return qa_chain

print("Initializing Chatbot.........")
llm = initialize_llm()

db_path = "/content/chroma_db"

if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
qa_chain = setup_qa_chain(vector_db, llm)

def calculate_empathy_score(response):
    """Analyze sentiment polarity using VADER for improved empathy scoring."""
    sentiment_scores = analyzer.polarity_scores(response)
    empathy_score = sentiment_scores["compound"]  # VADER provides a score b/w (-1 to +1)

    # Adjusting empathy scores for compassionate messages
    if "I'm here to listen" in response or "support you" in response:
        empathy_score += 0.2  # Boosing empathy scores for explicitly supportive messages
    elif empathy_score > 0:
        empathy_score += 0.1

    return max(-1, min(empathy_score, 1))  # Ensure it stays in range [-1, 1]

def save_feedback(user_input, chatbot_response, empathy_score, feedback):
    """Save user feedback to a log file."""
    feedback_entry = {
        "timestamp": str(datetime.now()),
        "user_input": user_input,
        "chatbot_response": chatbot_response,
        "empathy_score": empathy_score,
        "user_feedback": feedback
    }
    with open("chatbot_feedback.json", "a") as feedback_file:
        feedback_file.write(json.dumps(feedback_entry) + "\n")

def chatbot_response(user_input):
    if not user_input.strip():
        return "Please provide a valid input.", 0.0

    try:
        response = qa_chain.run(user_input)  # Generate chatbot response
        empathy_score = calculate_empathy_score(response)  #empathy score

        # Log chatbot interactions
        log_entry = {
            "timestamp": str(datetime.now()),
            "user_input": user_input,
            "chatbot_response": response,
            "empathy_score": empathy_score
        }
        with open("chatbot_logs.json", "a") as log_file:
            log_file.write(json.dumps(log_entry) + "\n")

        return response, empathy_score
    except Exception as e:
        print(f"Error in chatbot_response: {e}")
        return "An error occurred. Please try again later.", 0.0

def handle_feedback(user_input, chatbot_response, empathy_score, feedback):
    save_feedback(user_input, chatbot_response, empathy_score, feedback)
    return "Thank you for your feedback!"

# Gradio interface with feedback collection
with gr.Blocks(theme="Respair/Shiki@1.2.1") as app:
    gr.Markdown("#  AI-Therapist : a mental health chatbot ")
    gr.Markdown("A compassionate chatbot designed to assist with mental well-being. Please note: For serious concerns, contact a professional.")

    with gr.Row():
        chatbot_input = gr.Textbox(label="Ask a question:")
        chatbot_output = gr.Textbox(label="Chatbot Response", interactive=False)
        empathy_score_display = gr.Textbox(label="Empathy Score", interactive=False)

    feedback_radio = gr.Radio(["Yes", "No"], label="Was this response helpful?")
    feedback_button = gr.Button("Submit Feedback")
    feedback_output = gr.Textbox(label="Feedback Status", interactive=False)

    def chat_and_feedback(user_input):
        response, empathy_score = chatbot_response(user_input)
        return response, empathy_score

    chatbot_input.submit(chat_and_feedback, chatbot_input, [chatbot_output, empathy_score_display])
    feedback_button.click(handle_feedback, [chatbot_input, chatbot_output, empathy_score_display, feedback_radio], feedback_output)

    gr.Markdown("This chatbot provides general support. For urgent issues, seek help from licensed professionals.")

if vector_db is None:
    print("Error: vector_db is not initialized.")
else:
    print(f"Vector DB contains {vector_db._collection.count()} documents.")

app.launch(share=True, debug=True)


Initializing Chatbot.........


<ipython-input-13-721412bf8393>:33: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mode

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

ChromaDB created and data saved


theme_schema%401.2.1.json:   0%|          | 0.00/14.5k [00:00<?, ?B/s]

Vector DB contains 96 documents.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://13628186316668f71d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


<ipython-input-13-721412bf8393>:97: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(user_input)  # Generate chatbot response


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://13628186316668f71d.gradio.live


In [14]:
#empathy score calculation
import json

def load_chatbot_logs():
    """Load chatbot logs for evaluation."""
    try:
        with open("chatbot_logs.json", "r") as file:
            logs = [json.loads(line) for line in file.readlines()]
        return logs
    except FileNotFoundError:
        print("Error: chatbot_logs.json not found! Run the chatbot first.")
        return []

def evaluate_empathy():
    """Calculate average empathy score from chatbot responses."""
    logs = load_chatbot_logs()
    scores = [log["empathy_score"] for log in logs if "empathy_score" in log]

    if scores:
        avg_empathy_score = sum(scores) / len(scores)
        print(f"📊 Average Empathy Score: {avg_empathy_score:.2f} (Scale: -1 to +1)")
    else:
        print("No chatbot responses found for empathy evaluation.")

if __name__ == "__main__":
    evaluate_empathy()


📊 Average Empathy Score: 1.00 (Scale: -1 to +1)
